In [1]:
import torch
import brt.frontend.nn as nn
from brt.routers.app import RandScatterRouter, RandGatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandScatterRouter(dst_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = RandGatherRouter(dst_num=2)

    def forward(self, x):
        route_results = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x


class ThorMoE(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


thor_moe = ThorMoE()
thor_moe.cuda()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10).cuda()
x = thor_moe(x)
print(x)


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable